In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import time

### Importar tabela de dados e ajustar a variáve alvo

In [ ]:
tabela = pd.read_csv('../dados_coletados/tabela_modelagem.csv', na_values = '...')
tabela = tabela[tabela['Rendimento'] != '-']
tabela['Rendimento'] = tabela['Rendimento'].astype(float)
tabela = tabela[tabela['Rendimento'].notnull()]
tabela['Rendimento'] = tabela['Rendimento']**.5
tabela

### Dividir dados em teste e treino

In [ ]:
seed = 5744

In [ ]:
analisar = tabela.sample(frac = 1, random_state = seed)
analisar.head()

In [ ]:
analisar.info()

In [ ]:
X = analisar.iloc[:,11:]
y = analisar.iloc[:,8]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = seed)

In [ ]:
print('N treino:', len(X_train))
print('N teste:', len(X_test))

### Padronizar os atributos

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
scaler = sc.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
joblib.dump(scaler, '../resultados/scaler_reg.sav')

### Definir método para validação cruzada dos modelos de regressão

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

def regressor_cv(sample_method, estimator, X_train, y_train, parameters, cv = 5, n_iter = 10):
    if sample_method == 'RandomizedSearchCV':
        tunning = RandomizedSearchCV(
            estimator = estimator,
            param_distributions = parameters,
            n_iter = n_iter,
            cv = cv,
            scoring = 'neg_mean_squared_error',
            verbose = 3,
            random_state = seed,
            n_jobs = -1)
    elif sample_method == 'GridSearchCV': 
        tunning = GridSearchCV(
            estimator = estimator,
            param_grid = parameters,
            cv = cv,
            scoring = 'neg_mean_squared_error',
            verbose = 3,
            n_jobs = -1)
    else:
        print('Método de amostragem invalido! Escolha entre "RandomizedSearchCV" ou "GridSearchCV"')
    model = tunning.fit(X_train, y_train)
    display(model.best_params_)
    display(model.best_score_)

def resultados(y_pred):
    print('Erro absoluto médio:', mean_absolute_error(y_test, y_pred))
    print('Erro quadrado médio:', mean_squared_error(y_test, y_pred))
    print('Raiz do erro quadrado médio:', mean_squared_error(y_test, y_pred, squared = False))
    sns.scatterplot(
        x = y_test,
        y = y_pred)
    plt.xlabel('Observado')
    plt.ylabel('Predito')

### Regressão por Vetores Suporte

In [ ]:
from sklearn.svm import SVR

#### Modelo sem ajuste de hiperparametros

In [ ]:
regressor = SVR(kernel = 'rbf')
regressor.get_params()

In [ ]:
start = time.time()
regressor.fit(X_train, y_train)
end = time.time()
print('Finalizado em ', round(end - start, 1), ' s')

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
resultados(y_pred)

#### Seleção de hiperparametros

In [ ]:
C = [0.001 ,0.01, 0.1, 1, 10, 100] # Custo: trade-off entre margem mínima de separação e erro
gamma = [0.001, 0.01, 0.1, 1, 10, 100] # Curvatura dos vetores
epsilon = [0.1, 0.3, 0.5, 0.8, 1] # Tamanho da banda
kernel = ['rbf']

random_grid = {
    'C': C,
    'gamma': gamma,
    'epsilon': epsilon,
    'kernel': kernel}

In [ ]:
start = time.time()
regressor_cv('RandomizedSearchCV', SVR(), X_train, y_train, random_grid, cv = 5, n_iter = 60)
end = time.time()
print('Finalizado em ', round(end - start, 1), ' s')

##### Modelo final

In [ ]:
SVR_regressor = SVR(
    kernel = 'rbf',
    C = 10,
    gamma = 0.1,
    epsilon = 1)
SVR_regressor.fit(X_train, y_train)

In [ ]:
SVR_y_pred = SVR_regressor.predict(X_test)

In [ ]:
resultados(SVR_y_pred)

In [ ]:
SVR_savefile = '../resultados/SVR.sav'
joblib.dump(SVR_regressor, SVR_savefile)

#### Modelo salvo

In [ ]:
SVR_model = joblib.load('../resultados/SVR.sav')

In [ ]:
SVR_y_pred = SVR_model.predict(X_test)

In [ ]:
resultados(SVR_y_pred)

### Árvore de Decisão

In [ ]:
from sklearn.tree import DecisionTreeRegressor

#### Modelo sem ajuste de hiperparametros

In [ ]:
regressor = DecisionTreeRegressor(random_state = seed)
regressor.get_params()

In [ ]:
start = time.time()
regressor.fit(X_train, y_train)
end = time.time()
print('Finalizado em ', round(end - start, 1), ' s')

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
resultados(y_pred)

#### Seleção de hiperparametros

In [ ]:
splitter = ['best', 'random'] #Pontos de divisão criados aleatóriamente ('random') ou todos possíveis ('best')
max_features = [None, 2, 5, 10, 15] # Número máximo de atributos a cada decisão
max_depth = [None, 10, 100, 300, 600, 1000] # Tamanho máximo de uma árvore de decisão (número de nós)
min_samples_split = [2, 10, 50, 100] # Mínimo de amostras para criar uma divisão
min_samples_leaf = [1, 5, 20, 50] # Mínimo número de amostras para cada nó terminal
max_leaf_nodes = [None, 10, 100, 300, 600, 1000] # Número máximo de nós terminais (limita max_depth)

random_grid = {
    'splitter': splitter,
    'max_depth': max_depth,
    'min_samples_leaf': min_samples_leaf,
    'min_samples_split': min_samples_split,
    'max_features': max_features,
    'max_leaf_nodes': max_leaf_nodes}

In [ ]:
start = time.time()
regressor_cv('RandomizedSearchCV', DecisionTreeRegressor(), X_train, y_train, random_grid, cv = 5, n_iter = 500)
end = time.time()
print('Finalizado em ', round(end - start, 1), ' s')

##### Modelo final

In [ ]:
DT_regressor = DecisionTreeRegressor(
    random_state = seed,
    splitter = 'best',
    max_features = None,
    max_depth = 300,
    min_samples_split = 10,
    min_samples_leaf = 5,
    max_leaf_nodes = 300)
DT_regressor.fit(X_train, y_train)

In [ ]:
DT_y_pred = DT_regressor.predict(X_test)

In [ ]:
resultados(DT_y_pred)

In [ ]:
DT_savefile = '../resultados/DT_reg.sav'
joblib.dump(DT_regressor, DT_savefile)

#### Modelo salvo

In [ ]:
DT_model = joblib.load('../resultados/DT_reg.sav')

In [ ]:
DT_y_pred = DT_model.predict(X_test)

In [ ]:
resultados(DT_y_pred)

### Floresta Aleatória


In [ ]:
from sklearn.ensemble import RandomForestRegressor

#### Modelo sem ajuste de hiperparametros

In [ ]:
regressor = RandomForestRegressor(random_state = seed)
regressor.get_params()

In [ ]:
start = time.time()
regressor.fit(X_train, y_train)
end = time.time()
print('Finalizado em ', round(end - start, 1), ' s')

In [ ]:
y_pred = regressor.predict(X_test)
resultados(y_pred)

#### Seleção de hiperparametros

In [ ]:
n_estimators = [10, 100, 300, 1000] # Número de árvores de decisão
max_features = ['auto', 'sqrt', 'log2', 0.2, 0.5, 0.8] # Número de atributos a cada decisão
max_depth = [None, 10, 100, 300, 600, 1000] # Tamanho máximo de uma árvore de decisão (número de nós)
bootstrap = [True, False] # Método para selecionar amostras (com ou sem reposição)
min_samples_split = [2, 10, 50, 100] # Mínimo de amostras para criar uma divisão
min_samples_leaf = [1, 5, 20, 50] # Mínimo número de amostras para cada nó terminal
max_leaf_nodes = [None, 10, 100, 300, 600, 1000] # Número máximo de nós terminais (limita max_depth)

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
start = time.time()
regressor_cv('RandomizedSearchCV', RandomForestRegressor(), X_train, y_train, random_grid, cv = 3, n_iter = 60)
end = time.time()
print('Finalizado em ', round(end - start, 1), ' s')

##### Modelo final

In [ ]:
RF_regressor = RandomForestRegressor(
    n_estimators = 300,
    max_features = 0.5,
    max_depth = None,
    min_samples_split = 10,
    min_samples_leaf = 1,
    bootstrap = True,
    random_state = seed)
RF_regressor.fit(X_train, y_train)

In [ ]:
RF_y_pred = RF_regressor.predict(X_test)

In [ ]:
resultados(RF_y_pred)

In [ ]:
RF_savefile = '../resultados/rf_reg.sav'
joblib.dump(RF_regressor, RF_savefile)

#### Modelo salvo

In [ ]:
RF_load = joblib.load('../resultados/rf_reg.sav')

In [ ]:
RF_y_pred = RF_load.predict(X_test)

In [ ]:
resultados(RF_y_pred)

### Rede Neural

In [ ]:
#pip install keras-tuner

In [ ]:
import tensorflow as tf
from tqdm.keras import TqdmCallback

In [ ]:
import keras_tuner as kt

#### Seleção de hiperparametros

In [ ]:
def build_model(hp):
    model = tf.keras.models.Sequential()
    global X_train
    
    model.add(tf.keras.layers.Input(X_train.shape[1])) #Input layer
    
    for i in range(hp.Int("num_layers", 3, 12, 3)): #Hidden layers
        model.add(
            tf.keras.layers.Dense(
                # hiperparâmetros
                units = hp.Int(f"units_{i}", min_value = 32, max_value = 128, step = 32),
                activation = "relu"
                )
            )

    model.add(tf.keras.layers.Dense(1, activation = 'linear')) #Output layer
    
    model.compile(
        optimizer='adam', loss = 'mse', metrics=['mse', 'mae']
    )

    return model

In [ ]:
build_model(kt.HyperParameters())

In [ ]:
tuner = kt.RandomSearch(
    hypermodel = build_model,
    objective = 'val_mse',
    max_trials = 200,
    executions_per_trial = 1,
    overwrite = True,
    directory = '../resultados',
    project_name = 'kt_reg'
)

In [ ]:
start = time.time()
tuner.search(X_train, y_train,
             epochs = 10,
             batch_size = 2337,
             validation_split = 0.1,
             verbose = 0)
end = time.time()
print('Finalizado em ', round(end - start, 1), ' s')

#### Carregar resultados da seleção de modelos

In [ ]:
a_tuner = kt.RandomSearch(
    hypermodel = build_model,
    objective = 'val_mse',
    max_trials = 200,
    executions_per_trial = 1,
    overwrite = False,
    directory = '../resultados',
    project_name = 'kt_reg'
)

In [ ]:
a_tuner.results_summary()

In [ ]:
best = a_tuner.get_best_models(num_models = 1)
best = best[0]

In [ ]:
best.build(input_shape=(25))

In [ ]:
best.summary()

#### Treinar o modelo

In [ ]:
stop = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    patience = 5)

pasta = '../resultados/Rede_neural_reg'
arquivo = pasta + '/nn_model-{epoch:02d}-{val_loss:.2f}.hdf5'
cp = tf.keras.callbacks.ModelCheckpoint(
    arquivo,
    monitor = 'val_loss',
    verbose = 0,
    save_best_only = False,
    save_weights_only = False,
    save_frequency = 1)

In [ ]:
NN_model = best.fit(X_train, y_train,
                    batch_size = 2337,
                    epochs = 100,
                    validation_split = 0.2,
                    verbose = 0,
                    callbacks=[TqdmCallback(verbose = 1), cp, stop])

In [ ]:
hist = pd.DataFrame(NN_model.history)
hist['epoch'] = NN_model.epoch
hist

In [ ]:
def plot_history(hist):
    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Erro Absoluto Médio')
    plt.plot(hist['epoch'], hist['mae'],
            label='Treinamento')
    plt.plot(hist['epoch'], hist['val_mae'],
            label = 'Validação')
    plt.legend()

    plt.figure()
    plt.xlabel('Epoch')
    plt.ylabel('Erro Quadrado Médio')
    plt.plot(hist['epoch'], hist['mse'],
            label='Treinamento')
    plt.plot(hist['epoch'], hist['val_mse'],
            label = 'Validação')
    plt.legend()
    plt.show()

In [ ]:
plot_history(hist)

#### Modelo salvo

In [ ]:
load = tf.keras.models.load_model(
    '../resultados/Rede_neural_reg/nn_model-58-25.76.hdf5',
    compile = False)

In [ ]:
load.summary()

In [ ]:
NN_y_pred = load.predict(X_test)

In [ ]:
NN_y_pred = NN_y_pred.reshape(4676)

In [ ]:
resultados(NN_y_pred)

### XGBoost

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

#### Modelo sem ajuste de hiperparametros

In [ ]:
regressor = GradientBoostingRegressor()

In [ ]:
regressor.get_params()

In [ ]:
start = time.time()
regressor.fit(X_train, y_train)
end = time.time()
print('Finalizado em ', round(end - start, 1), ' s')

In [ ]:
y_pred = regressor.predict(X_test)

In [ ]:
resultados(y_pred)

#### Seleção de hiperparametros

In [ ]:
n_estimators = [10, 50, 100] # Número de árvores de decisão
max_features = ['auto'] # Número de atributos a cada decisão
max_depth = [None, 10, 100] # Tamanho máximo de uma árvore de decisão (número de nós)
max_leaf_nodes = [None] # Número máximo de nós terminais (limita max_depth)
min_samples_split = [2, 50] # Mínimo de amostras para criar uma divisão
min_samples_leaf = [1, 50] # Mínimo número de amostras para cada nó terminal
learning_rate = [0.1] # Taxa de aprendizado

random_grid = {
    'n_estimators': n_estimators,
    'max_features': max_features,
    'max_depth': max_depth,
    'max_leaf_nodes': max_leaf_nodes,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'learning_rate': learning_rate}

In [ ]:
start = time.time()
regressor_cv('GridSearchCV', GradientBoostingRegressor(), X_train, y_train, random_grid, cv = 3)
end = time.time()
print('Finalizado em ', round(end - start, 1), ' s')

##### Modelo final

In [ ]:
XGB_regressor = GradientBoostingRegressor(
    n_estimators = 100,
    max_features = 'auto',
    max_depth = 100,
    max_leaf_nodes = None,
    min_samples_split = 50,
    min_samples_leaf = 50,
    learning_rate = 0.1,
    verbose = 1,
    random_state = seed
)

In [ ]:
XGB_regressor.fit(X_train, y_train)

In [ ]:
XGB_y_pred = XGB_regressor.predict(X_test)

In [ ]:
resultados(XGB_y_pred)

In [ ]:
XGB_savefile = '../resultados/XGB_reg.sav'
joblib.dump(XGB_regressor, XGB_savefile)

#### Modelo salvo

In [ ]:
XGB_load = joblib.load('../resultados/XGB_reg.sav')

In [ ]:
XGB_y_pred = XGB_load.predict(X_test)

In [ ]:
resultados(XGB_y_pred)

### Comparação dos modelos

In [ ]:
SVR_mse = mean_squared_error(y_test, SVR_y_pred)
SVR_mae = mean_absolute_error(y_test, SVR_y_pred)
SVR_rmse = mean_squared_error(y_test, SVR_y_pred, squared = False)
DT_mse = mean_squared_error(y_test, DT_y_pred)
DT_mae = mean_absolute_error(y_test, DT_y_pred)
DT_rmse = mean_squared_error(y_test, DT_y_pred, squared = False)
RF_mse = mean_squared_error(y_test, RF_y_pred)
RF_mae = mean_absolute_error(y_test, RF_y_pred)
RF_rmse = mean_squared_error(y_test, RF_y_pred, squared = False)
NN_mse = mean_squared_error(y_test, NN_y_pred)
NN_mae = mean_absolute_error(y_test, NN_y_pred)
NN_rmse = mean_squared_error(y_test, NN_y_pred, squared = False)
XGB_mse = mean_squared_error(y_test, XGB_y_pred)
XGB_mae = mean_absolute_error(y_test, XGB_y_pred)
XGB_rmse = mean_squared_error(y_test, XGB_y_pred, squared = False)

In [ ]:
modelos_loss = {
    'Modelo': ['SVR', 'DT', 'RF', 'NN', 'XGB'],
    'mae': [SVR_mae, DT_mae, RF_mae, NN_mae, XGB_mae],
    'mse': [SVR_mse, DT_mse, RF_mse, NN_mse, XGB_mse],
    'rmse': [SVR_rmse, DT_rmse, RF_rmse, NN_rmse, XGB_rmse]
    }

In [ ]:
loss_tabela = pd.DataFrame(modelos_loss)
loss_tabela

In [ ]:
order = loss_tabela.sort_values('mse', ascending = False, inplace = False)
order

In [ ]:
sns.barplot(x = 'Modelo', y = 'mse',
            hue = None, data = loss_tabela,
            order = order['Modelo'], orient = None,
            color = None, palette = None, saturation=0.5)
plt.savefig('../resultados/mse_modelos.png', dpi = 300, bbox_inches = 'tight')